In [40]:
#Jenny Yu 4/25/2016

In [41]:
import numpy as np
import pandas as pd

In [42]:
train=pd.read_csv('train.csv',parse_dates=['Dates']) 
test=pd.read_csv('test.csv',parse_dates=['Dates']) 
#parse_dates to make sure the Dates column is handled as datetime63 dtype

In [44]:
import matplotlib.pyplot as plt
%matplotlib inline  

In [45]:
import seaborn as sns

In [47]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from matplotlib import cm

In [50]:
#Convert crime labels to label numbers named cData
CrimeData = preprocessing.LabelEncoder()
cData = CrimeData.fit_transform(train.Category)


In [51]:
# get binary vectors for months, districts, and hours.
month=pd.get_dummies(train.Dates.dt.month)
month2=pd.get_dummies(test.Dates.dt.month)
neighborhood = pd.get_dummies(train.PdDistrict)
neighborhood2 = pd.get_dummies(test.PdDistrict)
month.columns=['Jan','Feb','March','April','May','June','July','Aug','Sep','Oct','Nov','Dec']
month2.columns=['Jan','Feb','March','April','May','June','July','Aug','Sep','Oct','Nov','Dec']

In [53]:
months=month.columns.tolist()
hours=hour.columns.tolist()

# xtreme gradient boosting, followed by calibrated classifier

In [55]:
import os

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'

os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']



In [56]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.grid_search import GridSearchCV

In [57]:
train_data2 = pd.concat([ month,neighborhood], axis=1)
train_data2['crime']=cData
train_data2['Y']=train['Y']

In [60]:
hour_val = train.Dates.dt.hour
train_data2['hour_val']=hour_val
features2=months + ['BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION',
 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN', 'Y','hour_val']
training2, validation2= train_test_split(train_data2, train_size=.8, random_state=3)


In [61]:
clf = XGBClassifier(objective='multi:softprob', n_estimators=300, subsample=0.8, gamma=1.6, max_depth=6, nthread=4)
clf.fit(training2[features2],training2['crime'])


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=1.6, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=300, nthread=4,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8)

In [62]:
predict2=clf.predict_proba(validation2[features2])

In [65]:
log_loss(validation2['crime'], predict2)

2.4534932014742137

In [66]:
from sklearn.calibration import CalibratedClassifierCV
calib_iso=CalibratedClassifierCV(clf, cv=3, method='isotonic')
calib_iso.fit(training2[features2], training2['crime'])
prob_pos_iso= calib_iso.predict_proba(validation2[features2])




In [67]:

log_loss(validation2['crime'], prob_pos_iso) 

2.4452207884051957

In [68]:
#Build new array for test set
hour2=test.Dates.dt.hour
test_data=pd.concat([month2, neighborhood2], axis=1)
test_data['Y']=test['Y']
test_data['hour_val']=hour2


In [69]:
test_pred=calib_iso.predict_proba(test_data[features2])

In [70]:
test_pred=np.array(test_pred)
SF_pred=pd.DataFrame(test_pred, columns=CrimeData.classes_)
SF_pred.head()
     
        

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0.006617,0.137803,0.000049,0.000323,0.021616,0.000949,0.004993,0.050724,0.001972,0.000502,...,0.000332,0.003999,0.000363,0.039444,2.626915e-05,0.004028,0.084200,0.100839,0.051905,0.039927
1,0.007035,0.132623,0.000049,0.000502,0.017860,0.000941,0.003622,0.049450,0.001245,0.000441,...,0.000340,0.005412,0.000362,0.038907,2.235034e-05,0.004773,0.046174,0.109947,0.048364,0.034179
2,0.001570,0.055512,0.000121,0.000150,0.077284,0.002423,0.003795,0.017666,0.002828,0.000275,...,0.000064,0.005387,0.000360,0.031054,5.358938e-07,0.006386,0.069697,0.101087,0.028634,0.002898
3,0.003878,0.129370,0.000057,0.000320,0.015395,0.002273,0.003716,0.021092,0.003689,0.000269,...,0.000300,0.006067,0.000225,0.030951,6.262337e-06,0.003195,0.086048,0.161379,0.025831,0.016158
4,0.003878,0.129370,0.000057,0.000320,0.015395,0.002273,0.003716,0.021092,0.003689,0.000269,...,0.000300,0.006067,0.000225,0.030951,6.262337e-06,0.003195,0.086048,0.161379,0.025831,0.016158


In [71]:
SF_pred.to_csv('SF Prediction(3).csv', index = True, index_label = 'Id' )